In [1]:
import pandas as pd
from collections import Counter
from PIL import Image,ImageDraw,ImageFont
from scipy.stats import spearmanr
import os
import numpy as np
import colorsys

In [16]:
feattable = pd.read_csv("170728_2015-01-01_alphasort.csv")

In [17]:
feattable = feattable[feattable.feature_importance > 0.001125]

In [18]:
len(feattable)

80

In [19]:
feattable.reset_index(drop=True,inplace=True)

In [20]:
feattable.record_type.value_counts()

dispatch      44
ts            22
demarrests     5
arrests        5
shifts         3
ocag           1
Name: record_type, dtype: int64

#### Color settings

In [56]:
cathue = {
    'arrests':(102,194,165),
    'demarrests':(252,141,98),
    'dispatch':(141,160,203),
    'ocag':(231,138,195),
    'shifts':(166,216,84),
    'ts':(255,217,47)
    }

bkgdcolor = (255,255,255)
linecolor = (127,127,127)
textcolor = (64,64,64)

#### X

In [104]:
X = pd.read_csv("170728_2015-01-01_test-matrix-and-labels-and-scores.csv")

In [105]:
X = X.sample(n=int(len(X)/float(4)))

In [106]:
len(X)

389

#### Mapping saturation to data values

In [107]:
from scipy.stats import percentileofscore as pct

In [108]:
Xpct = X.drop(['officer_id','as_of_date','outcome','score'],axis=1)

In [109]:
Xpct = Xpct.loc[:,feattable.feature]

In [110]:
def norm(col):
    rng = max(col) - min(col)
    tmp = [(item - min(col)) / float(rng) for item in col]
    return [int(round(item*100)) for item in tmp]

In [111]:
counter=-1
for col in Xpct.columns:
    counter+=1
    if np.random.binomial(1,0.01)==1:
        print counter
    if all(X[col]<=0):
        Xpct[col] = 0
    elif 'ocnd' in col:
        for i in X.index:
            if X[col].loc[i]==1:
                Xpct[col].loc[i] = 65
            elif X[col].loc[i]==0:
                Xpct[col].loc[i] = 25
    else:
        Xpct[col] = norm(X[col])

14


In [112]:
rectwidth = 32
rectheight = 32
px_w = len(feattable) * rectwidth
px_h = len(Xpct) * rectheight

In [113]:
X.sort_values(by='score',ascending=False,inplace=True)

In [114]:
def offtablefeat(X,feattable):
    canvas = Image.new('RGB',(px_w,px_h),bkgdcolor)
    feats = feattable.feature
    
    counter = -1
    for i in X.index:
        counter+=1
        ycoord = counter * rectheight
        
        for feat in feats:
            tmp = feattable[feattable.feature==feat]
            record_type = tmp.record_type.iloc[0]
            idx = tmp.index[0]
            
            xcoord = idx * rectwidth
            bbox = [xcoord, ycoord, xcoord + rectwidth, ycoord + rectheight]
            
            alpha = int( Xpct[feat].loc[i] / float(100) * 255 )
            fillcolor = cathue[record_type] + (alpha,)
            
            draw = ImageDraw.Draw(canvas,'RGBA')
            draw.rectangle(bbox,fill=fillcolor,outline=None)
            
    return canvas

In [115]:
im = offtablefeat(X,feattable)

In [141]:
w = im.width
h = im.height
x = 75
canvas = Image.new('RGB',((w+x),(h)),bkgdcolor)
canvas.paste(im,(x,0))

In [142]:
draw = ImageDraw.Draw(canvas)

counter = -1
for i in X.index:
    counter+=1
    score = int( X.score.loc[i] * 100 )
    text = str(score)
    font = ImageFont.truetype('VeraMono.ttf', rectheight-1)
    fontWidth, fontHeight = font.getsize(text)
    
    xpos = 5
    ypos = counter * rectheight
    draw.text((xpos,ypos),text=text,font=font,fill=textcolor)
    
    linex = w+x
    liney = counter * rectheight
    draw.line([(0,liney),(linex,liney)],width=1,fill=linecolordark)

In [143]:
w = canvas.width
h = canvas.height
y = 750
unicanvas = Image.new('RGB',((w),(h+y)),bkgdcolor)
unicanvas.paste(canvas,(0,y))
canvas = None

In [144]:
draw = ImageDraw.Draw(unicanvas)
for i in feattable.index:
    feature = feattable.featstr.loc[i]
    record_type = feattable.record_type.loc[i]
    text = str(feature)
    
    font = ImageFont.truetype('VeraMono.ttf', rectwidth - 7)
    fontWidth, fontHeight = font.getsize(text)
    
    imtmp = Image.new('RGBA', (fontWidth + 4, fontHeight), cathue[record_type] + (100,))
    drawtmp = ImageDraw.Draw(imtmp)
    drawtmp.text((3,0),text=text,font=font,fill=textcolor)
    imtmp = imtmp.rotate(90,expand=1)
    
    xpos = i * rectwidth + x - 5
    ypos = y - imtmp.height - 12
    unicanvas.paste(imtmp,(xpos,ypos),imtmp)
    
    linex = i * rectwidth + x
    liney = y
    draw.line([(linex,liney),(linex,unicanvas.height)],width=1,fill=linecolordark)

In [145]:
unicanvas.save("/Users/damoncrockett/Desktop/table_view.png")